In [23]:
import requests
import bs4
import numpy as np
import pandas as pd
import datetime
import pymysql

#connect to MySQL database
conn = pymysql.connect(host='nbastats.chp0zhljxmaa.eu-west-1.rds.amazonaws.com', port=3306, user='tpauley', passwd='Rp44906255$1', db='nbastats')
#create connection cursor
cur = conn.cursor()

#get text from HTML document as response
response = requests.get('http://www.rotowire.com/daily/NBA/optimizer.php?site=FanDuel')

#setup BeaufifulSoup 
soup = bs4.BeautifulSoup(response.text,"lxml")


#find table in HTML by looking for 'tr' table row objects
table_body = soup.find_all('tr')

#build blank lists to be filled in
l_names = []
l_tm = []
l_opp = []
l_pos = []
l_sal = []
l_mins = []
l_pts = []
date = datetime.datetime.today().strftime("%m_%d_%Y")

#loop through all rows in HTML (table_body find all 'td')
for rows in table_body[2:]:
    col = rows.find_all('td')
    
    #find name and replace GTD and special characters
    nm = col[1].text.strip().replace("\nGTD", "").lower().replace(".","")
    tm = col[2].text.strip()
    opp = col[3].text.strip()
    pos = col[4].text.strip()
    
    #find mins, salary, and points and convert to floats
    mins = float(col[7].text.strip().replace(",", ""))
    sal = float(col[8].attrs.get("data-salary").replace(",", ""))
    pts = float(col[9].attrs.get("data-points").replace(",", ""))
    
    #append each of the columns to their respective lists
    l_names.append(nm)
    l_tm.append(tm)
    l_opp.append(opp)
    l_pos.append(pos)
    l_sal.append(sal)
    l_mins.append(mins)
    l_pts.append(pts)

#build dataframe from lists
pd_data = pd.DataFrame(
    {'name': l_names,
     'team': l_tm,
     'pos': l_pos,
     'opp': l_opp,
     'sal': l_sal,
     'mins': l_mins,
     'pts': l_pts,
     'date' : date
    })


#old system to CSV
#pd_data.to_csv(date + '_fd_lineup.csv')

#iterate through dataframe to insert new records to lineup_fd table
for index, row in pd_data.iterrows():
        
        #cur.execute('''INSERT into nbastats.lineup_fd (date, mins, name, opp, pos, pts, sal, team)
         #         values (str_to_date(%s,'%%m_%%d_%%Y'), %s, %s, %s, %s, %s, %s, %s)''',
          #        (row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]))

       # conn.commit()
        
#close database connection        
cur.close()
conn.close() 
print("done")

IntegrityError: (1062, "Duplicate entry 'russell westbrook' for key 'name_date'")